In [2]:
import openpyxl   
import pandas as pd
import numpy as np
from pandas import read_excel
import matplotlib.pyplot as plt
import scipy.optimize as scp

In [51]:
raw_data=pd.read_excel("spark2.xlsx", engine = 'openpyxl') 
data = pd.DataFrame(raw_data)
data = data.drop([0,1]).reset_index(drop=True)
data

,Unnamed: 0,Sample,T [s],ch1 [V],ch2 [V],mean [V],inverted [V]
0,2,-163,-0.000003,0.022033,0.000000,0.000285,-0.000285
1,3,-162,-0.000003,0.021032,0.000595,0.000284,-0.000284
2,4,-161,-0.000003,0.020030,0.000496,0.000257,-0.000257
3,5,-160,-0.000003,0.020330,0.000595,0.000281,-0.000281
4,6,-159,-0.000003,0.021632,0.000496,0.000270,-0.000270
...,...,...,...,...,...,...,...
131065,131067,130902,0.002618,0.020230,0.000694,0.000268,-0.000268
131066,131068,130903,0.002618,0.020931,0.000496,0.000254,-0.000254
131067,131069,130904,0.002618,0.021332,0.000297,0.000260,-0.000260
131068,131070,130905,0.002618,0.021833,0.000595,0.000264,-0.000264


In [52]:
time_column = data.columns[2]
ch1_column = data.columns[3]
data_column = data.columns[-1]
print(time_column,ch1_column,data_column)
#print(np.average(data[ch1_column])*10)

%matplotlib widget
plt.plot(data[time_column],data[ch1_column])
plt.plot(data[time_column],data[data_column], color='r')
plt.show()


T [s] ch1 [V] inverted [V]


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [53]:
tester = []
for i in range(len(data[ch1_column])):
    tester.append(data[ch1_column].to_numpy()[i]-data[ch1_column].to_numpy()[i-1])

thershold =  (np.max(tester)-np.average(tester))/3.5
print(thershold)

%matplotlib widget
plt.plot(data[time_column], tester)
plt.axhline(thershold, color='r')
#plt.plot(data[time_column],data[data_column], color='r')

0.010701766857142858


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [6]:
def f(x,a,b):
    return a*np.exp(-x/b)
    

In [54]:
points_candidates = np.argwhere(tester>thershold)
points_candidates = np.append(points_candidates,[[(len(data[ch1_column])-1)]], axis=0)
#points_candidates.append()

i = 0
points = []
while i<(len(points_candidates)-1):
    distance = points_candidates[i+1][0]-points_candidates[i][0]
    if distance < 200:
        if data[ch1_column][points_candidates[i][0]]>data[ch1_column][points_candidates[i+1][0]]:
            points.append(points_candidates[i][0])
        else:
            points.append(points_candidates[i+1][0])
        i = i + 1

    else:
        points.append(points_candidates[i][0])

    i = i + 1

points.append(len(data[ch1_column])-1)
#print(points)
#print(points_candidates)


%matplotlib widget
plt.plot(data[time_column],data[data_column])


for i in range(len(points)):
    plt.axvline(x= data[time_column][points[i]], color = 'r')

"""

for i in range(len(points_candidates)):
    plt.axvline(x= data[time_column][points_candidates[i][0]], color = 'g')

"""

plt.show()




Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [55]:

thaus = []

#for i in [10]:

for i in range(len(points)-1):

    indexes = np.arange(points[i],points[i+1])[:-20]
    val_between_imp = data[data_column].to_numpy()[indexes]
    peak = np.argmax(val_between_imp)

    times_fit = np.array(data[time_column ].to_numpy()[indexes][peak:] - data[time_column ].to_numpy()[indexes][peak:][0],dtype=np.float32)
    val_fit = np.array((data[data_column].to_numpy()[indexes][peak:]-np.average(data[data_column].to_numpy()[indexes][peak:][-20:])),dtype=np.float32)
    a_init = val_fit[0]
    b_init = 0.00000014
    #b_init = -times_fit[80]/np.log(val_fit[80]/a_init)

    params, pcov = scp.curve_fit(f, times_fit, val_fit, [a_init,b_init])
    err = np.sqrt(np.diag(pcov))

    thaus.append(params[1])
    #print(params[1])
    
    #"""
    """
    %matplotlib widget
    plt.plot(times_fit,val_fit)
    plt.plot(times_fit, f(times_fit, params[0], params[1]), color = 'r')
    """
    #"""
    
print(thaus)
print(np.average(thaus),np.std(thaus))


[1.1847562268843742e-07, 1.0521484099262636e-07, 1.1346075824847887e-07, 1.136553495543135e-07, 1.2363322577421e-07, 1.3249612455421193e-07, 1.6273956000518737e-07, 1.4924289089236446e-07, 1.3380770493838007e-07, 1.1541285496592428e-07, 1.242002361730842e-07, 1.1818895799721504e-07, 1.3465196492152588e-07, 1.5929002640491458e-07, 1.685591478368551e-07, 1.3532102318113035e-07, 1.363450748149595e-07, 1.809601176428315e-07, 1.409435200012185e-07, 1.8444777786618108e-07, 1.8712326495820216e-07, 1.5130323989777616e-07, 1.1914862357369606e-07, 1.3182707189930242e-07, 1.725054469601303e-07, 1.7492913249621895e-07, 1.945667612534219e-07, 1.9251926617158737e-07]
1.4553462809515665e-07 2.6910637686926914e-08


nitrogen:
2.3585663246765498e-06 6.909764575890499e-08
air1:
2.324143345682654e-06 7.554586521535033e-08
air2:
2.3452365742603783e-06 5.3840971863432014e-08
crone1:
1.2384908420188277e-06 8.084168262122479e-08
crone2:
5.165562047605518e-07 2.63762452029122e-08
crone3:
2.0042874543094457e-06 5.207545170862822e-08
spark1:
2.2626358380928828e-07 9.484269041802186e-08
spark2:
1.4553462809515665e-07 2.6910637686926914e-08


